In [1]:
import pandas as pd

In [2]:
train_set = pd.read_json('train_filtered_nl2bash.json', lines=True)
train_set

,invocation,cmd,cmdset,cmdcath
0,"Copy loadable kernel module ""mymodule.ko"" to t...",sudo cp mymodule.ko /lib/modules/$(uname -r)/k...,"[sudo, uname]","[SYSINFO, SECOP]"
1,"Display all lines containing ""IP_MROUTE"" in th...",cat /boot/config-`uname -r` | grep IP_MROUTE,"[cat, grep, uname]","[SYSINFO, TXTPROC]"
2,Display current running kernel's compile-time ...,cat /boot/config-`uname -r`,"[cat, uname]","[SYSINFO, TXTPROC]"
3,"Find all loadable modules for current kernel, ...",find /lib/modules/`uname -r` -regex .*perf.*,"[find, uname]","[SYSINFO, FDOPS]"
4,"Look for any instance of ""HIGHMEM"" in the curr...","grep ""HIGHMEM"" /boot/config-`uname -r`","[grep, uname]","[SYSINFO, TXTPROC]"
...,...,...,...,...
8867,using exec in find command to dispaly the sear...,find . ... -exec cat {} \; -exec echo \;,[find],[FDOPS]
8868,verbosely create intermediate directoriy tmp a...,mkdir -pv /tmp/boostinst,[mkdir],[FDOPS]
8869,view the manual page of find,man find,[man],[HELPDOC]
8870,"wait 2 seconds and then print ""hello""","echo ""hello `sleep 2 &`""","[echo, sleep]","[SHUTIL, PROCCTRL]"


In [3]:
import yaml

In [46]:
with open('categories.yml') as f:
    categories_list = yaml.safe_load(f)
cat_description = '; '.join([c['description'] for c in categories_list])
categories_dict = {c['name']:c for c in categories_list}

with open('tools_docs.yml') as f:
    tools_short_doc = yaml.safe_load(f)

In [25]:
all_tools_cat = []
for c in categories_list:
    all_tools_cat += c['utils']

In [26]:
set(all_tools_cat) == set(tools_short_doc.keys())

True

In [49]:
with open('tools_cheatsheets.yml') as f:
    tools_long_doc = yaml.safe_load(f)

In [55]:
def make_simple_prediction(record):
    return f'<user>: {record.invocation}\n'\
           f'<IDE-genie>: {record.cmd}'

def make_cmd_cat_prediction(record):
    selected_categories = ", ".join(record.cmdcath)
    
    return f'<user>: {record.invocation}\n'\
           f'<category encyclopedia>: {cat_description}\n'\
           f'<logic>[Medium:success]: <|{selected_categories}|>'

def make_cmd_tools_prediction(record):
    tools_proposal = []
    for c in record.cmdcath:
        tools_proposal += categories_dict[c]['utils']
    tools_description = '; '.join([f'{t}: {tools_short_doc[t]}' for t in tools_proposal])
    selected_tools = ", ".join(record.cmdset)
    
    selected_categories = ", ".join(record.cmdcath)
    return f'<user>: {record.invocation}\n'\
           f'<category logic>[trivial:success]: <|{selected_categories}|>\n'\
           f'<tools encyclopedia>: {tools_description}\n'\
           f'<tool logic>[medium:success]: <|{selected_tools}|>'

def make_cmd_usage_prediction(record):
    tools_cheatsheets = []
    tools_description = '\n\n'.join([f'{tools_long_doc[t]}' for t in record.cmdset])
    
    selected_tools = ", ".join(record.cmdset)
    selected_categories = ", ".join(record.cmdcath)
    return f'<user>: {record.invocation}\n'\
           f'<category logic>[trivial:success]: <|{selected_categories}|>\n'\
           f'<tool logic>[medium:success]: <|{selected_tools}|>\n'\
           f'<cheatsheets>: {tools_description}\n'\
           f'<IDE-genie>: {record.cmd}' 

In [57]:
print(make_simple_prediction(train_set.loc[10]))

<user>: Add "execute" to the permissions of all directories in the home directory tree
<IDE-genie>: find ~ -type d -exec chmod +x {} \;


In [58]:
print(make_cmd_cat_prediction(train_set.loc[10]))

<user>: Add "execute" to the permissions of all directories in the home directory tree
<category encyclopedia>: "FDOPS (File and Directory Operations)": Creating, deleting, modifying files and directories; "TXTPROC (Text Processing)": Manipulating or displaying text data; "SYSINFO (System Information and Management)": Providing system info or managing resources; "ARCH (File Compression and Archiving)": Compressing, decompressing, or managing archives; "NET (Networking)": Handling networking tasks; "PROCCTRL (Job and Process Control)": Managing running processes and jobs; "HELPDOC (Help and Documentation)": Providing help, manuals, or documentation; "PROGSCRIPT (Programming and Scripting)": Used for programming tasks or script execution; "SHUTIL (Shell Utilities)": Providing various utility functionalities; "SECOP (Secure Operations)": Managing security or access control
<logic>[Medium:success]: <|FDOPS|>


In [59]:
print(make_cmd_tools_prediction(train_set.loc[10]))

<user>: Add "execute" to the permissions of all directories in the home directory tree
<category logic>[trivial:success]: <|FDOPS|>
<tools encyclopedia>: basename: returns non-directory portion of a pathname; cd: changes the current directory; ln: creates a link to a file; mkdir: creates a new directory; rmdir: removes empty directories; rm: removes files or directories; cp: copies files or directories; mv: moves or renames files or directories; touch: changes file timestamps; ls: lists directory contents; find: searches for files in a directory hierarchy; pwd: prints current working directory; pushd: adds a directory to the stack and navigates to it; popd: removes a directory from the stack and navigates to it; dirname: returns directory portion of a pathname; tree: lists contents of directories in a tree-like format; rename: renames files
<tool logic>[medium:success]: <|find|>


In [60]:
print(make_cmd_usage_prediction(train_set.loc[10]))

<user>: Add "execute" to the permissions of all directories in the home directory tree
<category logic>[trivial:success]: <|FDOPS|>
<tool logic>[medium:success]: <|find|>
<cheatsheets>: # find
# Find files or directories under the given directory tree, recursively.
# More information: .
# Find files by extension:
find root_path -name '*.ext'
# Find files matching multiple path/name patterns:
find root_path -path '**/path/**/*.ext' -or -name '*pattern*'
# Find directories matching a given name, in case-insensitive mode:
find root_path -type d -iname '*lib*'
# Find files matching a given pattern, excluding specific paths:
find root_path -name '*.py' -not -path '*/site-packages/*'
# Find files matching a given size range, limiting the recursive depth to "1"::
find root_path -maxdepth 1 -size +500k -size -10M
# Run a command for each file (use `{}` within the command to access the filename):
find root_path -name '*.ext' -exec wc -l {} \;
# Find files modified in the last 7 days:
find root_